In [ ]:
# Dependencies
!pip3 install pytorch_lightning pytorch-nlp torch torchvision torchaudio transformers sentencepiece accelerate --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.4/716.4 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 

In [ ]:
from transformers import T5Tokenizer, T5EncoderModel
import torch
import re
from torch.utils.data import DataLoader, RandomSampler
import os
import pandas as pd
import numpy as np

import pytorch_lightning as pl
from torchnlp.encoders import LabelEncoder
from torchnlp.datasets.dataset import Dataset
from torchnlp.utils import collate_tensors

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using device: {}".format(device))

Using device: cuda:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
cur_dir = './drive/MyDrive/AIfSR'

Mounted at /content/drive


In [ ]:
# Load ProtT5 in half-precision (more specifically: the encoder-part of ProtT5-XL-U50 in half-precision)

transformer_link = "Rostlab/prot_t5_xl_half_uniref50-enc"
print("Loading: {}".format(transformer_link))
model = T5EncoderModel.from_pretrained(transformer_link)

# *What does full-precision mean and why only use it on cpu?
model.full() if device=='cpu' else model.half() # only cast to full-precision if no GPU is available
model = model.to(device)
model = model.eval()
tokenizer = T5Tokenizer.from_pretrained(transformer_link, do_lower_case=False )

Loading: Rostlab/prot_t5_xl_half_uniref50-enc


In [ ]:
for p in model.parameters():
  print(p.shape)

torch.Size([128, 1024])
torch.Size([4096, 1024])
torch.Size([4096, 1024])
torch.Size([4096, 1024])
torch.Size([1024, 4096])
torch.Size([32, 32])
torch.Size([1024])
torch.Size([16384, 1024])
torch.Size([1024, 16384])
torch.Size([1024])
torch.Size([4096, 1024])
torch.Size([4096, 1024])
torch.Size([4096, 1024])
torch.Size([1024, 4096])
torch.Size([1024])
torch.Size([16384, 1024])
torch.Size([1024, 16384])
torch.Size([1024])
torch.Size([4096, 1024])
torch.Size([4096, 1024])
torch.Size([4096, 1024])
torch.Size([1024, 4096])
torch.Size([1024])
torch.Size([16384, 1024])
torch.Size([1024, 16384])
torch.Size([1024])
torch.Size([4096, 1024])
torch.Size([4096, 1024])
torch.Size([4096, 1024])
torch.Size([1024, 4096])
torch.Size([1024])
torch.Size([16384, 1024])
torch.Size([1024, 16384])
torch.Size([1024])
torch.Size([4096, 1024])
torch.Size([4096, 1024])
torch.Size([4096, 1024])
torch.Size([1024, 4096])
torch.Size([1024])
torch.Size([16384, 1024])
torch.Size([1024, 16384])
torch.Size([1024])
torch

In [ ]:
# This cell shows how to extract embedded layers.

# fake data from the notebook example
sequence_examples = ["PRTEINO", "SEQWENCE"]

# **Not sure what this is for. Another good thing to circle back with the 
#   collaborator about. Do we need to be concerned with "rare/ambiguous" amino acids?
#   Are there other edge cases or sections of data that we'll need to think
#   about when preparing the data?
# this will replace all rare/ambiguous amino acids by X and introduce white-space between all amino acids
sequence_examples = [" ".join(list(re.sub(r"[UZOB]", "X", sequence))) for sequence in sequence_examples]


# tokenize sequences and pad up to the longest sequence in the batch
# *What are special tokens here?
# *Look into the batch encode plus method - how does it work. Should be able to explain it to soemone else
ids = tokenizer.batch_encode_plus(sequence_examples, add_special_tokens=True, padding="longest") # this padding option is great/convenient
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)


# generate embeddings
with torch.no_grad():
    embedding_repr = model(input_ids=input_ids,attention_mask=attention_mask)

# **Look into this architecture more. I don't understand why they are pulling the dimensions they are pulling (i.e. [0,:7] and [1,:8])
# extract embeddings for the first ([0,:]) sequence in the batch while removing padded & special tokens ([0,:7]) 
emb_0 = embedding_repr.last_hidden_state[0,:7] # shape (7 x 1024)
print(f"Shape of per-residue embedding of first sequences: {emb_0.shape}")

# *Based on this and the previous two lines, it seems like you can ignore padding
#  and special tokens by taking only up to :7. I don't fully understand this - how does it work?
# do the same for the second ([1,:]) sequence in the batch while taking into account different sequence lengths ([1,:8])
emb_1 = embedding_repr.last_hidden_state[1,:8] # shape (8 x 1024)

# if you want to derive a single representation (per-protein embedding) for the whole protein
emb_0_per_protein = emb_0.mean(dim=0) # shape (1024)

print(f"Shape of per-protein embedding of first sequences: {emb_0_per_protein.shape}")

Shape of per-residue embedding of first sequences: torch.Size([7, 1024])
Shape of per-protein embedding of first sequences: torch.Size([1024])


In [ ]:
# Next Steps:
# We need to fine-tune this model to predict protein-family. 
#   We can use an existing dataset for this but it should be one that either has
#   a large variety of proteins labeled by protein family, or else have a lot of
#   proteins belonging to families relevant for this task. My guess is we'll want
#   a datest with a larger variety because we probably don't know which families are relevant for this task
# 
# With the ProtTrans architecutre for pre-trained models it will be easy to extract the vector reprenstations
#
# Next, how can we map those representations into a vector space. I don't have much experience with this - may take more research.
#
# Test the architecuture: are we able to see that the vector representation of test 
# data proteins cluster closely with other proteins in the same family? 
#
# Finally, find where the vector representations of the unknown proteins land in the vector space.

In [ ]:
class PfamData():

  def __init(self, data_path):
    super(PfamData, self).__init__()
    self.data_path = data_path
    self.train_data = self.load_data('train')
    self.test_data = self.load_data('test')
    self.label_encoder(self.train_data)

  def load_data(self, phase):
    data = []
    for fn in os.listdir(os.path.join(self.data_path, phase)):
      with open(os.path.join(self.data_path, phase, fn)) as f:
        data.append(pd.read_csv(f, index_col=None, encoding='utf-8'))
    data = pd.concat(data)

    seq = list(data['sequence'])
    label = list(data['family_accession'])

    seq = [" ".join("".join(sample.split())) for sample in seq]
    seq = [re.sub(r"[UZOB]", "X", sample) for sample in seq]

    assert len(seq) == len(label)

    # Do I need the collate? What is this for?
    # return Dataset(self.collate_lists(seq, label))
    return Dataset(seq, label)

  def label_encoder(self, d):
    labels = list(pd.unique(d['family_accession']))
    self.label_size = len(labels)

    # Label Encoder
    self.label_encoder = LabelEncoder(labels, reserved_labels=[])
    self.label_encoder.unknown_index = None


In [ ]:
# Fine-tuned ProtT5 model

class ProtT5Classifier(pl.LightningModule):

  def __init__(self, hparams):
    super(ProtT5Classifier, self).__init__()
    # The author of the model I'm basing this on uses an argument parser. Not sure we need this level of utility - can probably just be a params dict
    self.hparams = hparams
    self.batch_size = self.hparams.batch_size

    self.data = PfamData(os.path.join(cur_dir, 'pfam'))

    self.metric = pl.metrics.sklearns.Accuracy()
    self.loss = torch.nn.CrossEntropyLoss()

    self._build_model()

    # freeze layers
    for param in self.model.parameters()[:-1]:
      param.require_grad = False

  def _build_model(self):
    transformer_link = "Rostlab/prot_t5_xl_half_uniref50-enc"
    self.model = T5EncoderModel.from_pretrained(transformer_link)
    print("Loading: {}".format(transformer_link))

    # *What does full-precision mean and why only use it on cpu?
    model.full() if device=='cpu' else model.half() # only cast to full-precision if no GPU is available

    self.encoder_features = 1024 # embedding layer. this should be the size of the second to last layer assuming it is a FC layer

    tokenizer = T5Tokenizer.from_pretrained(transformer_link, do_lower_case=False )

    # Classification head
    self.classification_head = torch.nn.Sequential(
        torch.nn.Linear(self.encoder_features*4, self.data.label_encoder.vocab_size),
        torch.nn.Tanh(),
    )

    def predict(self, sample: dict) -> dict:
      """ Predict function.
      :param sample: dictionary with the text we want to classify.
      Returns:
          Dictionary with the input text and the predicted label.
      """
      if self.training:
          self.eval()

      with torch.no_grad():
          model_input, _ = self.prepare_sample([sample], prepare_target=False)
          model_out = self.forward(**model_input)
          logits = model_out["logits"].numpy()
          predicted_labels = [
              self.label_encoder.index_to_token[prediction]
              for prediction in np.argmax(logits, axis=1)
          ]
          sample["predicted_label"] = predicted_labels[0]

      return sample
  
    def pool_strategy(self, features,
                      pool_cls=True, pool_max=True, pool_mean=True,
                      pool_mean_sqrt=True):
        token_embeddings = features['token_embeddings']
        cls_token = features['cls_token_embeddings']
        attention_mask = features['attention_mask']

        ## Pooling strategy
        output_vectors = []
        if pool_cls:
            output_vectors.append(cls_token)
        if pool_max:
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            token_embeddings[input_mask_expanded == 0] = -1e9  # Set padding tokens to large negative value
            max_over_time = torch.max(token_embeddings, 1)[0]
            output_vectors.append(max_over_time)
        if pool_mean or pool_mean_sqrt:
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)

            #If tokens are weighted (by WordWeights layer), feature 'token_weights_sum' will be present
            if 'token_weights_sum' in features:
                sum_mask = features['token_weights_sum'].unsqueeze(-1).expand(sum_embeddings.size())
            else:
                sum_mask = input_mask_expanded.sum(1)

            sum_mask = torch.clamp(sum_mask, min=1e-9)

            if pool_mean:
                output_vectors.append(sum_embeddings / sum_mask)
            if pool_mean_sqrt:
                output_vectors.append(sum_embeddings / torch.sqrt(sum_mask))

        output_vector = torch.cat(output_vectors, 1)
        return output_vector

    def forward(self, input_ids, token_type_ids, attention_mask):
        """ Usual pytorch forward function.
        :param tokens: text sequences [batch_size x src_seq_len]
        :param lengths: source lengths [batch_size]
        Returns:
            Dictionary with model outputs (e.g: logits)
        """
        input_ids = torch.tensor(input_ids, device=self.device)
        attention_mask = torch.tensor(attention_mask,device=self.device)

        word_embeddings = self.model(input_ids)[0]

        pooling = self.pool_strategy({"token_embeddings": word_embeddings,
                                      "cls_token_embeddings": word_embeddings[:, 0],
                                      "attention_mask": attention_mask,
                                      })
        return {"logits": self.classification_head(pooling)}

    # def prepare_sample(self, sample: list, prepare_target: bool = True) -> (dict, dict):
    #   """
    #   Function that prepares a sample to input the model.
    #   :param sample: list of dictionaries.
      
    #   Returns:
    #       - dictionary with the expected model inputs.
    #       - dictionary with the expected target labels.
    #   """
    #   sample = collate_tensors(sample)

    #   inputs = self.tokenizer.batch_encode_plus(sample["seq"],
    #                                             add_special_tokens=True,
    #                                             padding=True,
    #                                             truncation=True,
    #                                             max_length=self.hparams.max_length)

    #   if not prepare_target:
    #       return inputs, {}

    #   # Prepare target:
    #   try:
    #       targets = {"labels": self.label_encoder.batch_encode(sample["label"])}
    #       return inputs, targets
    #   except RuntimeError:
    #       print(sample["label"])
    #       raise Exception("Label encoder found an unknown label.")

    def training_step(self, batch: tuple, batch_nb: int, *args, **kwargs) -> dict:
      """ 
      Runs one training step. This usually consists in the forward function followed
          by the loss function.
      
      :param batch: The output of your dataloader. 
      :param batch_nb: Integer displaying which batch this is
      Returns:
          - dictionary containing the loss and the metrics to be added to the lightning logger.
      """
      inputs, targets = batch
      model_out = self.forward(**inputs)
      loss_val = self.loss(model_out, targets)

      tqdm_dict = {"train_loss": loss_val}
      output = OrderedDict(
          {"loss": loss_val, "progress_bar": tqdm_dict, "log": tqdm_dict})

      # can also return just a scalar instead of a dict (return loss_val)
      return output

    def test_step(self, batch: tuple, batch_nb: int, *args, **kwargs) -> dict:
        """ Similar to the training step but with the model in eval mode.
        Returns:
            - dictionary passed to the validation_end function.
        """
        inputs, targets = batch
        model_out = self.forward(**inputs)
        loss_test = self.loss(model_out, targets)

        y = targets["labels"]
        y_hat = model_out["logits"]
        
        labels_hat = torch.argmax(y_hat, dim=1)
        test_acc = self.metric_acc(labels_hat, y)
        
        output = OrderedDict({"test_loss": loss_test, "test_acc": test_acc,})

        return output

    def train_dataloader(self) -> DataLoader:
        """ Function that loads the train set. """
        self._train_dataset = self.__retrieve_dataset(val=False, test=False)
        return DataLoader(
            dataset=self._train_dataset,
            sampler=RandomSampler(self._train_dataset),
            batch_size=self.hparams.batch_size,
            collate_fn=self.prepare_sample,
            num_workers=self.hparams.loader_workers,
        )

    # def test_dataloader(self) -> DataLoader:
    #     """ Function that loads the validation set. """
    #     self._test_dataset = self.__retrieve_dataset(train=False, val=False)
    #     return DataLoader(
    #         dataset=self._test_dataset,
    #         batch_size=self.hparams.batch_size,
    #         collate_fn=self.prepare_sample,
    #         num_workers=self.hparams.loader_workers,
    #     )